In [ ]:
R.version.string


In [ ]:
library(mgcv)
library(gratia)
library(tidyr)
getwd()

In [ ]:
# Get current working directory
current_path <- getwd()

# Get the parent directory
data_path <- dirname(current_path)

# Build the full file path
file_path <- file.path(data_path, "data", "step1_cbsa.csv")

# Read the CSV file
df <- read.csv(file_path)


In [ ]:

detect_outlier <- function(x) {
  
  # calculate first quantile
  Quantile1 <- quantile(x, probs=.25)
  
  # calculate third quantile
  Quantile3 <- quantile(x, probs=.75)
  
  # calculate inter quartile range
  IQR = Quantile3-Quantile1
  
  # return true or false
  #x > Quantile3 + (IQR*1.5) | x < Quantile1 - (IQR*1.5)
  x > Quantile3 + (IQR*1.5) | x < Quantile1 - (IQR*1.5)

}

# create remove outlier function
remove_outlier <- function(dataframe,
                           columns=names(dataframe)) {
  
  # for loop to traverse in columns vector
  for (col in columns) {
    
    # remove observation if it satisfies outlier function
    dataframe <- dataframe[!detect_outlier(dataframe[[col]]), ]
  }
  # return dataframe
  print("Remove outliers")
  print(dataframe)
}


In [ ]:
names(df)

# df all year

In [ ]:
# Create subfolder to save GAM results
output_dir <- "GAM_results"
if (!dir.exists(output_dir)) {
  dir.create(output_dir)
}
df$tmp_c <- (df$tmp - 32) * 5 / 9

# Filter the data for the specific years
#df_year_all <- df[df$accident_year %in% c(2012,2013, 2014, 2015,2016, 2017,2018, 2019,2020,2021, 2022), ]
df_year_all <- df[df$accident_year %in% c(2012, 2014,2016,2018,2020, 2022), ]
# Filter out rows with Total_Events2 < 3
df_year_all <- df_year_all[df_year_all$Total_Events2 >= 3, ]

# Convert relevant columns to factors
df_year_all$HHS_Region <- as.factor(df_year_all$HHS_Region)

# Calculate `pf` and `build_year_new`
df_year_all$pf <- df_year_all$Total_Events2 / df_year_all$Est_YRB_Built_total_units_list
df_year_all$build_year_new <- df_year_all$build_time_1980_to_later_list
df_year_all$pf <- df_year_all$pf * 100000 * 12

# Remove outliers from specific columns
outlier_columns <- c(
  'Black.Alone', 'pf', 'Pct_SA_62_and_over_list', 'Industrial',
  'Transportation.and.storage', 'Pct_EDU_Bachelor_or_higher_list',
  'Pct_HOU_Occupied_units_list', 'build_year_new', 'zndx',
  'POPPCT_URBAN', 'tmp_c'
)
for (col in outlier_columns) {
  df_year_all <- remove_outlier(df_year_all, c(col))
}

# Fit GAM model
k_value <- 3
mod_all <- gam(
  pf ~ factor(HHS_Region) +
    s(POPPCT_URBAN, k = k_value) +
    s(Pct_HOU_Occupied_units_list, k = k_value) +
    s(build_year_new, k = k_value) +
    s(Pct_EDU_Bachelor_or_higher_list, k = k_value) +
    s(Pct_SA_62_and_over_list, k = k_value) +
    s(Black.Alone, k = k_value) +
    s(Industrial, k = k_value) +
    s(Transportation.and.storage, k = k_value) +
    s(tmp_c, k = k_value) +
    s(zndx, k = k_value),
  data = df_year_all,
  family = Gamma(link = "log"),
  select = TRUE
)

# Save model summary
summary_output <- capture.output(summary(mod_all))
summary_file <- file.path(output_dir, "step_1_results_final_remove_outlier_cm_density_all_year.txt")
writeLines(summary_output, summary_file)

# Save concurvity diagnostics
concurvity_output <- capture.output(concurvity(mod_all))
concurvity_file <- file.path(output_dir, "concurvity_density_all_year.txt")
writeLines(concurvity_output, concurvity_file)

# Save compare smooths result
comp <- compare_smooths(mod_all, mod_all)
rest <- unnest(comp, data)
csv_file <- file.path(output_dir, "step_1_results_final_remove_outlier_cm_density_all_year.csv")
write.csv(rest, csv_file, row.names = FALSE)

# Save GAM plot
plot_file <- file.path(output_dir, "plot_mod_density_all_year.png")
png(plot_file, width = 1200, height = 800)
plot(mod_all, page = 1)
dev.off()

# Save GAM diagnostics plot
gam_check_plot_file <- file.path(output_dir, "gam_check_plots_density_all_year.png")
png(gam_check_plot_file, width = 800, height = 800)
gam.check(mod_all, rep = 100, page = 1)
dev.off()

# df season

In [ ]:
# Define a function to map months to seasons
get_season <- function(month) {
  if (month %in% c(12, 1, 2)) {
    return("Winter")
  } else if (month %in% c(3, 4, 5)) {
    return("Spring")
  } else if (month %in% c(6, 7, 8)) {
    return("Summer")
  } else if (month %in% c(9, 10, 11)) {
    return("Autumn")
  }
}
df$tmp_c <- (df$tmp - 32) * 5 / 9

# Add a season column to the dataframe
df$season <- sapply(df$accident_month, get_season)

# Create output directory if not exists
output_dir <- "GAM_results"
if (!dir.exists(output_dir)) {
  dir.create(output_dir)
}

# Iterate over seasons
seasons <- c("Winter", "Spring", "Summer", "Autumn")

for (season in seasons) {
  
  # Filter the data for the specific season
  df_season <- df[df$season == season, ]
  
  # Filter out rows with Total_Events2 < 3
  df_season <- df_season[df_season$Total_Events2 >= 3, ]
  
  # Skip empty dataframes
  if (nrow(df_season) == 0) {
    message(paste("Skipping", season, "- no data after filtering."))
    next
  }

  # Convert relevant columns to factors
  df_season$HHS_Region <- as.factor(df_season$HHS_Region)
  
  # Calculate the `pf` column
  df_season$pf <- df_season$Total_Events2 / df_season$Est_YRB_Built_total_units_list
  df_season$pf <- df_season$pf * 100000 * 12
  df_season$build_year_new <- df_season$build_time_1980_to_later_list

  # Remove outliers for specific columns
  outlier_columns <- c(
    'Black.Alone', 'pf', 'Pct_SA_62_and_over_list', 'Industrial',
    'Transportation.and.storage', 'Pct_EDU_Bachelor_or_higher_list',
    'Pct_HOU_Occupied_units_list', 'build_year_new', 'zndx',
    'POPPCT_URBAN', 'tmp_c'
  )
  for (col in outlier_columns) {
    df_season <- remove_outlier(df_season, c(col))
  }
  
  # Re-check if any rows remain
  if (nrow(df_season) == 0) {
    message(paste("Skipping", season, "- no data after outlier removal."))
    next
  }
  
  # Define the GAM model
  k_value = 3
  mod_all <- gam(
    pf ~ factor(HHS_Region) +
      s(POPPCT_URBAN, k = k_value) +
      s(Pct_HOU_Occupied_units_list, k = k_value) +
      s(build_year_new, k = k_value) +
      s(Pct_EDU_Bachelor_or_higher_list, k = k_value) +
      s(Pct_SA_62_and_over_list, k = k_value) +
      s(Black.Alone, k = k_value) +
      s(Industrial, k = k_value) +
      s(Transportation.and.storage, k = k_value) +
      s(tmp_c, k = k_value) +
      s(zndx, k = k_value),
    data = df_season,
    family = Gamma(link = "log"),
    select = TRUE
  )
  
  # Summarize the model
  summary_output <- capture.output(summary(mod_all))
  writeLines(summary_output, file.path(output_dir, paste0("step_1_results_final_remove_outlier_cm_density_", season, ".txt")))
  
  # Save the concurvity
  concurvity_output <- capture.output(concurvity(mod_all))
  writeLines(concurvity_output, file.path(output_dir, paste0("concurvity_density_", season, ".txt")))
  
  # Save smooth comparisons
  comp <- compare_smooths(mod_all, mod_all)
  rest <- unnest(comp, data)
  write.csv(rest, file.path(output_dir, paste0("step_1_results_final_remove_outlier_cm_density_", season, ".csv")), row.names = FALSE)
  
  # Save the GAM results plot
  png(file.path(output_dir, paste0("plot_mod_all_density_", season, ".png")), width = 1200, height = 800)
  plot(mod_all, page = 1)
  dev.off()
  
  # Save GAM diagnostic plots
  png(file.path(output_dir, paste0("gam_check_plots_all_density_", season, ".png")), width = 800, height = 800)
  gam.check(mod_all, rep = 100, page = 1)
  dev.off()
}


# df region

In [ ]:
# Create output directory if not exists
output_dir <- "GAM_results"
if (!dir.exists(output_dir)) {
  dir.create(output_dir)
}
df$tmp_c <- (df$tmp - 32) * 5 / 9

# Get unique regions
regions <- unique(df$Census_Region)
regions <- regions[!is.na(regions)]  # Remove NA if present

for (region in regions) {
  
  # Filter the data for the specific region
  df_region <- df[df$Census_Region == region, ]
  
  # Filter out rows with Total_Events2 < 3
  df_region <- df_region[df_region$Total_Events2 >= 3, ]
  
  # Skip empty dataframes
  if (nrow(df_region) == 0) {
    message(paste("Skipping", region, "- no data after filtering."))
    next
  }

  # Convert relevant columns to factors
  df_region$HHS_Region <- as.factor(df_region$HHS_Region)
  
  # Calculate the `pf` column
  df_region$pf <- df_region$Total_Events2 / df_region$Est_YRB_Built_total_units_list
  df_region$pf <- df_region$pf * 100000 * 12
  df_region$build_year_new <- df_region$build_time_1980_to_later_list

  # Remove outliers for specific columns
  outlier_columns <- c(
    'Black.Alone', 'pf', 'Pct_SA_62_and_over_list', 'Industrial',
    'Transportation.and.storage', 'Pct_EDU_Bachelor_or_higher_list',
    'Pct_HOU_Occupied_units_list', 'build_year_new', 'zndx',
    'POPPCT_URBAN', 'tmp_c'
  )
  for (col in outlier_columns) {
    df_region <- remove_outlier(df_region, c(col))
  }
  
  # Re-check if any rows remain
  if (nrow(df_region) == 0) {
    message(paste("Skipping", region, "- no data after outlier removal."))
    next
  }
  
  # Define the GAM model
  k_value = 3
  mod_all <- gam(
    pf ~ factor(HHS_Region) +
      s(POPPCT_URBAN, k = k_value) +
      s(Pct_HOU_Occupied_units_list, k = k_value) +
      s(build_year_new, k = k_value) +
      s(Pct_EDU_Bachelor_or_higher_list, k = k_value) +
      s(Pct_SA_62_and_over_list, k = k_value) +
      s(Black.Alone, k = k_value) +
      s(Industrial, k = k_value) +
      s(Transportation.and.storage, k = k_value) +
      s(tmp_c, k = k_value) +
      s(zndx, k = k_value),
    data = df_region,
    family = Gamma(link = "log"),
    select = TRUE
  )

  
  # Summarize the model
  summary_output <- capture.output(summary(mod_all))
  writeLines(summary_output, file.path(output_dir, paste0("step_1_results_final_remove_outlier_cm_density_", region, ".txt")))
  
  # Save the concurvity
  concurvity_output <- capture.output(concurvity(mod_all))
  writeLines(concurvity_output, file.path(output_dir, paste0("concurvity_density_", region, ".txt")))
  
  # Save smooth comparisons
  comp <- compare_smooths(mod_all, mod_all)
  rest <- unnest(comp, data)
  write.csv(rest, file.path(output_dir, paste0("step_1_results_final_remove_outlier_cm_density_", region, ".csv")), row.names = FALSE)
  
  # Save the GAM results plot
  png(file.path(output_dir, paste0("plot_mod_all_density_", region, ".png")), width = 1200, height = 800)
  plot(mod_all, page = 1)
  dev.off()
  
  # Save GAM diagnostic plots
  png(file.path(output_dir, paste0("gam_check_plots_all_density_", region, ".png")), width = 800, height = 800)
  gam.check(mod_all, rep = 100, page = 1)
  dev.off()
}
